In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data = pd.read_csv("clean_data.csv")
data.head()

,Unnamed: 0,Label,Text
0,0,Politics,budget to set scene for election gordon brown ...
1,1,Politics,army chiefs in regiments decision military chi...
2,3,Politics,observers to monitor uk election ministers wil...
3,4,Politics,kilroy names election seat target exchat show ...
4,5,Politics,donor attacks blairbrown feud the reported feu...


In [2]:
data.replace({0:"Politics", 1:"Sport", 2:"Technology", 3:"Entertainment", 4:"Business"}, inplace=True)
data.Label.value_counts()

Label
Sport            505
Business         503
Politics         403
Entertainment    369
Technology       347
Name: count, dtype: int64

In [3]:
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

In [4]:
X_train, X_val, y_train, y_val = train_test_split(data["Text"], data["Label"], test_size=0.2, random_state=42)